In [28]:
import pandas as pd

SEARCH_LOG_FILE_PATH = "logs/resultsLog.tsv"
USER_LOG_FILE_PATH = "logs/userActionLog.tsv"
SEARCH_LOG_FILE = pd.read_csv(SEARCH_LOG_FILE_PATH, sep='\t')
USER_LOG_FILE = pd.read_csv(USER_LOG_FILE_PATH, sep='\t')

def write_search_log():
    with open(SEARCH_LOG_FILE_PATH,'w') as write_tsv:
        write_tsv.write(SEARCH_LOG_FILE.to_csv(sep='\t', index=False))

def write_user_log():
    with open(USER_LOG_FILE_PATH,'w') as write_tsv:
        write_tsv.write(USER_LOG_FILE.to_csv(sep='\t', index=False))

def get_new_session_id():
    return SEARCH_LOG_FILE.iloc[-1, 0] + 1

In [29]:
get_new_session_id()

3

In [30]:
# this is the class for the thing returned to the front end
class search_results_data:
    def __init__(self, search_session_id, query_input, ranks, qids, pids, query_labels, passages, scores):
        """
        search_session_id - intiger, unique for each search session
        query_input - string, the query
        all of the below should be of the same length and in order
        ranks - list of ranks
        qids - list of qids
        pids - list of pids
        query_labels - list of label queries
        passages - list of passages
        scores - list of scores from search matching
        """
        assert len(ranks)==len(qids)==len(pids)==len(query_labels)==len(passages)==len(scores)
        table = []
        for i, rank in enumerate(ranks):
            table.append({
                "rank": ranks[i],
                "qid": qids[i],
                "pid": pids[i],
                "query_label": query_labels[i],
                "passage": passages[i],
                "score": scores[i],
            })
        self.search_session_id = search_session_id
        self.query_input = query_input
        self.table = table

dummy_result = search_results_data(
        search_session_id = 29,
        query_input = "alksjdflj",
        ranks = [1,2,3], 
        qids = [123, 324, 348], 
        pids = [15324, 124, 642], 
        query_labels = ["柯南国语版全集", "柯南", "违章查询"], 
        passages = ["名侦探柯南 国语版-动漫动画-全集高清正版视频在线观看-爱奇艺", "名侦探柯南国语版全集(国语) 名侦探柯南国语版动漫全集 在线观看 - 哈哈动漫网", "名侦探柯南(国语)-名侦探柯南(国语)全集(1-837共1000集)-动画片 - 搜狐视频"], 
        scores = [0.231, 0.123, 0.095]
    )

In [37]:
def update_search_log(result_object):
    global SEARCH_LOG_FILE
    # The useful things are:
    # result_object.search_session_id
    # result_object.query_input
    # result_object.table
    for item in result_object.table:
        SEARCH_LOG_FILE = SEARCH_LOG_FILE.append({"search_session_id": result_object.search_session_id, "query": result_object.query_input, "rank": item["rank"], "qid": item["pid"], "pid": item["pid"], "score": item["score"]} , ignore_index=True)
    write_search_log()
    
update_search_log(dummy_result)

{'rank': 1, 'qid': 123, 'pid': 15324, 'query_label': '柯南国语版全集', 'passage': '名侦探柯南 国语版-动漫动画-全集高清正版视频在线观看-爱奇艺', 'score': 0.231}
{'rank': 2, 'qid': 324, 'pid': 124, 'query_label': '柯南', 'passage': '名侦探柯南国语版全集(国语) 名侦探柯南国语版动漫全集 在线观看 - 哈哈动漫网', 'score': 0.123}
{'rank': 3, 'qid': 348, 'pid': 642, 'query_label': '违章查询', 'passage': '名侦探柯南(国语)-名侦探柯南(国语)全集(1-837共1000集)-动画片 - 搜狐视频', 'score': 0.095}


In [39]:
def update_user_log(sessionID, actionType, pid, rank):
    global USER_LOG_FILE
    USER_LOG_FILE = USER_LOG_FILE.append({"search_session_id": sessionID, "action_type": actionType, "pid": pid, "rank": rank} , ignore_index=True)
    write_user_log()
    
update_user_log(123, "upvote", 234, 1)